In [1]:
import json
import numpy as np
import pandas as pd
import pickle

In [2]:
with open('seasons.pickle') as f:
    seasons = pickle.load(f)
for year, season in seasons.iteritems():
    print year,
    season.load_scores()

2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018


In [3]:
with open('pd_data/name_fixes_all.json') as f:
    all_name_fixes = pickle.load(f)

IOError: [Errno 2] No such file or directory: 'pd_data/name_fixes_all.json'

In [ ]:
elt_df_list = []
comp_df_list = []
for season in (seasons['2017'], seasons['2018']):
    for event in season.events:
        for discipline in event.disciplines:
            for segment in discipline.segments:
                for scorecard in segment.scorecards:
                    for element in scorecard.elements:
                        elt_df_list.append({
                            'skater': all_name_fixes.get(scorecard.skater.name, scorecard.skater.name),
                            'country': scorecard.skater.country,
                            'segment_name': event.name + '_' + segment.name,
                            'season': season.champ_year,
                            'discipline': discipline.discipline.name,
                            'segment_rank': scorecard.rank,
                            'start_order': scorecard.starting_number,
                            'elt_number': element.number,
                            'elt_name': element.name,
                            'elt_info': element.info,
                            'elt_bv': element.base_value,
                            'elt_bonus': element.bonus,
                            'elt_finalgoe': element.goe,
                            'elt_goes': ','.join(map(str, element.goes)),
                            'elt_points': element.points
                        })
                    for comp in scorecard.components:
                        comp_df_list.append({
                            'skater': all_name_fixes.get(scorecard.skater.name, scorecard.skater.name),
                            'country': scorecard.skater.country,
                            'segment_name': event.name + '_' + segment.name,
                            'season': season.champ_year,
                            'discipline': discipline.discipline.name,
                            'segment_rank': scorecard.rank,
                            'start_order': scorecard.starting_number,
                            'comp_name': all_name_fixes.get(comp.name, comp.name),
                            'comp_scores': ','.join(map(str, comp.scores)),
                            'comp_points': comp.points
                        })

In [ ]:
elts_df = pd.DataFrame(elt_df_list)
comp_df = pd.DataFrame(comp_df_list)

In [ ]:
def remove_mr_ms(judge):
    judge = judge.replace(r'\xa0', '')
    judge = judge.replace('Mr. ', '')
    judge = judge.replace('Mr ', '')
    judge = judge.replace('Ms. ', '')
    judge = judge.replace('Ms ', '')
    judge = judge.replace('Mrs. ', '')
    return judge

In [ ]:
panels = {}  # map segment name to judging panel
# judge_names = set()
for season in seasons.values():
    for event in season.events:
        for discipline in event.disciplines:
            for segment in discipline.segments:
                if 'Function' in segment.panel.officials:
                    del segment.panel.officials['Function']
                    print event, segment
                panel = {}
                for function, official in segment.panel.officials.iteritems():
                    name = remove_mr_ms(official.name)
                    name = all_name_fixes.get(name, name)
                    panel[function] = name
                    if len(name) < 5:
                        print event, segment, name
#                     judge_names.add(name)
                panels[event.name + '_' + segment.name] = panel

In [4]:
judges = {k: ','.join([v['Judge No.{0}'.format(judge_num+1)] for judge_num in xrange(len(v)-6)])
          for k, v in panels.iteritems()}

NameError: name 'panels' is not defined

In [43]:
elts_df['judges'] = elts_df.apply(lambda row: judges[row.segment_name], axis=1)
comp_df['judges'] = comp_df.apply(lambda row: judges[row.segment_name], axis=1)

In [51]:
elts_df['elt_goes'] = elts_df.apply(lambda row: row.elt_goes if row.segment_name != 'gpusa2016_pairs_short'
              else ','.join([goe for goe in row.elt_goes.split(',') if goe != '-']), axis=1)

In [54]:
elts_df[elts_df.elt_goes.str.contains('-')]

,country,discipline,elt_bonus,elt_bv,elt_finalgoe,elt_goes,elt_info,elt_name,elt_number,elt_points,season,segment_name,segment_rank,skater,start_order,judges
0,JPN,men,False,12.30,-0.51,"-1.0,0.0,0.0,-1.0,-1.0,0.0,1.0,-1.0,0.0",,4F,1,11.79,2017,gpusa2016_men_short,1,Shoma UNO,9,"Cynthia BENSON,Lisa JELINEK,Maira ABASOVA,Fran..."
1,JPN,men,False,13.30,-4.00,"-3.0,-3.0,-3.0,-3.0,-3.0,-2.0,-3.0,-3.0,-3.0",<,4T+3T<,2,9.30,2017,gpusa2016_men_short,1,Shoma UNO,9,"Cynthia BENSON,Lisa JELINEK,Maira ABASOVA,Fran..."
4,JPN,men,True,9.35,-0.14,"1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0",,3A,5,9.21,2017,gpusa2016_men_short,1,Shoma UNO,9,"Cynthia BENSON,Lisa JELINEK,Maira ABASOVA,Fran..."
14,USA,men,False,10.30,-4.00,"-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0",,4T,1,6.30,2017,gpusa2016_men_short,3,Jason BROWN,8,"Cynthia BENSON,Lisa JELINEK,Maira ABASOVA,Fran..."
23,CAN,men,False,2.80,0.07,"0.0,0.0,0.0,0.0,-1.0,1.0,1.0,0.0,0.0",,FCSp3,3,2.87,2017,gpusa2016_men_short,4,Nam NGUYEN,5,"Cynthia BENSON,Lisa JELINEK,Maira ABASOVA,Fran..."
28,RUS,men,False,10.30,-4.00,"-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0",,4T,1,6.30,2017,gpusa2016_men_short,5,Sergei VORONOV,4,"Cynthia BENSON,Lisa JELINEK,Maira ABASOVA,Fran..."
38,USA,men,True,9.13,-2.00,"-3.0,-3.0,-3.0,-3.0,-2.0,-2.0,-3.0,-3.0,-3.0",!,3F+3T<,4,7.13,2017,gpusa2016_men_short,6,Timothy DOLENSKY,3,"Cynthia BENSON,Lisa JELINEK,Maira ABASOVA,Fran..."
44,BEL,men,False,3.50,-0.09,"0.0,1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0",,CCoSp4,3,3.41,2017,gpusa2016_men_short,7,Jorik HENDRICKX,7,"Cynthia BENSON,Lisa JELINEK,Maira ABASOVA,Fran..."
45,BEL,men,True,11.33,-1.40,"-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-3.0,-2.0",,3Lz+3T,4,9.93,2017,gpusa2016_men_short,7,Jorik HENDRICKX,7,"Cynthia BENSON,Lisa JELINEK,Maira ABASOVA,Fran..."
46,BEL,men,True,5.61,-0.30,"0.0,0.0,0.0,0.0,-1.0,-1.0,1.0,-1.0,-1.0",,3Lo,5,5.31,2017,gpusa2016_men_short,7,Jorik HENDRICKX,7,"Cynthia BENSON,Lisa JELINEK,Maira ABASOVA,Fran..."


In [53]:
for i, row in elts_df.iterrows():
    goes = row.elt_goes.split(',')
    if '-' in goes

ValueError: ("invalid literal for int() with base 10: '-'", u'occurred at index 0')